# 군집화 (Clustering)

**학습 목표:**
- 정답(label)이 없는 데이터에서 숨겨진 구조나 그룹을 찾아내는 **비지도 학습**의 한 종류인 군집화를 학습합니다.
- 거리 기반 군집화 알고리즘의 대표인 **K-평균(K-means)**을 사용하여, 최적의 군집 수(K)를 찾는 **엘보우 방법(Elbow Method)**을 적용해봅니다.
- 밀도 기반 군집화 알고리즘인 **DBSCAN**을 사용하여, 복잡한 모양의 군집을 찾고 노이즈를 식별하는 방법을 배웁니다.
- 군집화 성능을 평가하는 지표인 **실루엣 계수(Silhouette Score)**를 이해하고 계산해봅니다.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score

### (1) 데이터 준비: 가상 고객 데이터 생성
`make_blobs` 함수를 사용하여 군집 분석에 적합한 가상 데이터를 생성합니다. 이 데이터는 고객의 '연간 소득'과 '소비 점수'를 나타낸다고 가정합니다.

In [ ]:
X, y_true = make_blobs(n_samples=300, centers=4, cluster_std=0.9, random_state=0)
X_scaled = StandardScaler().fit_transform(X)

plt.figure(figsize=(8, 6))
plt.scatter(X_scaled[:, 0], X_scaled[:, 1], s=50)
plt.title('Simulated Customer Data')
plt.xlabel('Annual Income (Scaled)')
plt.ylabel('Spending Score (Scaled)')
plt.show()

### (2) K-평균 (K-means) 군집화
K-평균은 가장 널리 사용되는 군집화 알고리즘 중 하나입니다. 사용자가 군집의 수(K)를 미리 지정하면, 알고리즘이 각 데이터 포인트를 K개의 군집 중 하나에 할당합니다.

#### 최적의 K 찾기: 엘보우 방법(Elbow Method)
K값을 1부터 늘려가며 각 K에 대한 군집 내 오차 제곱합(Inertia)을 계산합니다. Inertia 값이 급격히 줄어들다가 완만해지는 지점, 즉 그래프의 '팔꿈치(elbow)'에 해당하는 K를 최적의 군집 수로 선택합니다.

In [ ]:
inertia = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(range(1, 11), inertia, marker='o', linestyle='--')
plt.title('Elbow Method for Optimal K')
plt.xlabel('Number of clusters (K)')
plt.ylabel('Inertia')
plt.xticks(range(1, 11))
plt.grid(True)
plt.show()
print("그래프에서 K=4 지점에서 팔꿈치가 형성되는 것을 볼 수 있습니다.")

In [ ]:
# K=4로 K-means 모델 최종 학습 및 시각화
kmeans = KMeans(n_clusters=4, random_state=42, n_init='auto')
y_kmeans = kmeans.fit_predict(X_scaled)

plt.figure(figsize=(10, 7))
sns.scatterplot(x=X_scaled[:, 0], y=X_scaled[:, 1], hue=y_kmeans, palette='viridis', s=50, legend='full')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=200, alpha=0.9, marker='X', label='Centroids')
plt.title('K-means Clustering (K=4)')
plt.xlabel('Annual Income (Scaled)')
plt.ylabel('Spending Score (Scaled)')
plt.legend()
plt.show()

### (3) 군집화 성능 평가: 실루엣 계수 (Silhouette Score)
실루엣 계수는 각 데이터 포인트가 자신의 군집에 얼마나 잘 속해 있고, 다른 군집과는 얼마나 잘 분리되어 있는지를 나타내는 지표입니다. -1에서 1 사이의 값을 가지며, 1에 가까울수록 군집화가 잘 되었다고 평가합니다.

In [ ]:
score = silhouette_score(X_scaled, y_kmeans)
print(f"Silhouette Score for K=4: {score:.4f}")

### (4) DBSCAN (Density-Based Spatial Clustering of Applications with Noise)
DBSCAN은 데이터의 밀도를 기반으로 군집을 형성합니다. K-평균과 달리 군집의 수를 미리 지정할 필요가 없고, 기하학적으로 복잡한 모양의 군집도 잘 찾아내며, 어떤 군집에도 속하지 않는 노이즈 포인트를 구별해내는 장점이 있습니다.

In [ ]:
# eps: 이웃을 찾기 위한 거리, min_samples: 핵심 포인트를 정의하기 위한 최소 이웃 수
dbscan = DBSCAN(eps=0.5, min_samples=5)
y_dbscan = dbscan.fit_predict(X_scaled)

plt.figure(figsize=(10, 7))
sns.scatterplot(x=X_scaled[:, 0], y=X_scaled[:, 1], hue=y_dbscan, palette='viridis', s=50, legend='full')
plt.title('DBSCAN Clustering')
plt.xlabel('Annual Income (Scaled)')
plt.ylabel('Spending Score (Scaled)')
plt.show()

# DBSCAN 결과 분석 (노이즈는 -1로 레이블링됨)
n_clusters = len(set(y_dbscan)) - (1 if -1 in y_dbscan else 0)
n_noise = list(y_dbscan).count(-1)
print(f"Estimated number of clusters: {n_clusters}")
print(f"Estimated number of noise points: {n_noise}")